In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.metrics import  r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder

In [4]:
df = pd.read_csv('./drive/MyDrive/train.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
features_with_nan = [feature for feature in df. columns if df[feature].isnull().sum()>1]

In [6]:
for feature in features_with_nan:
    print(feature, np.round(df[feature].isnull().mean()*100, 2), '% missing')

floor 62.99 % missing
osm_city_nearest_population 0.02 % missing
reform_house_population_1000 5.22 % missing
reform_house_population_500 9.73 % missing
reform_mean_floor_count_1000 5.97 % missing
reform_mean_floor_count_500 10.78 % missing
reform_mean_year_building_1000 5.8 % missing
reform_mean_year_building_500 10.59 % missing
street 0.57 % missing


In [7]:
X_df, y_series = df.drop(['floor', 'per_square_meter_price'], axis=1), df['per_square_meter_price']

In [8]:
numerical_feat=[features for features in df.columns if df[features].dtypes!='O']
print('The number of numerical features are', len(numerical_feat))

The number of numerical features are 70


In [9]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [10]:
df_dropped = df.drop(features_with_nan, axis = 1)

In [11]:
cat_features = []
for i in df_dropped.columns:
    if i not in numerical_feat:
        cat_features.append(i)

In [12]:
df_dropped = df_dropped.drop(cat_features, axis = 1)

In [13]:
scaler = StandardScaler()
support_scaled = scaler.fit_transform(df_dropped)

In [14]:
clust = KMeans()

clst_labels = clust.fit_predict(support_scaled)
labels_set = list(set(clst_labels))
print(labels_set)

[0, 1, 2, 3, 4, 5, 6, 7]


In [17]:
clst_labels.shape

(279792,)

In [22]:
X_df['cluster'] = clst_labels

In [23]:
features_nan_num = []
for i in features_with_nan:
  if i in numerical_feat:
    features_nan_num.append(i)

In [30]:
features_with_nan.remove('floor')

In [34]:
X_df.groupby('cluster')['cluster']

In [35]:
sub_df = X_df[X_df[features_with_nan].isnull()]

In [39]:
for drop in features_nan_num:
  sub_df = X_df[X_df[drop].isnull()]
  for clust in sub_df.groupby('cluster')['cluster'].unique():
    similar_val_list = X_df[X_df['cluster'] == clust[0]][drop]
    if len(similar_val_list) > 1:
      missed_val = similar_val_list.dropna().mean()
      X_df.loc[similar_val_list[similar_val_list.isnull()].index, drop] = missed_val

In [41]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279792 entries, 0 to 279791
Data columns (total 76 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   city                                 279792 non-null  object 
 1   id                                   279792 non-null  object 
 2   lat                                  279792 non-null  float64
 3   lng                                  279792 non-null  float64
 4   osm_amenity_points_in_0.001          279792 non-null  int64  
 5   osm_amenity_points_in_0.005          279792 non-null  int64  
 6   osm_amenity_points_in_0.0075         279792 non-null  int64  
 7   osm_amenity_points_in_0.01           279792 non-null  int64  
 8   osm_building_points_in_0.001         279792 non-null  int64  
 9   osm_building_points_in_0.005         279792 non-null  int64  
 10  osm_building_points_in_0.0075        279792 non-null  int64  
 11  osm_building_

In [43]:
features_nan_cat = [feature for feature in X_df. columns if X_df[feature].isnull().sum()>1]

In [44]:
features_nan_cat

['street']

In [47]:
len(X_df['street'].unique())

28842

In [56]:
X_df['street'] = X_df['street'].str[1:]

In [57]:
X_df['street']

0         27289
1         17052
2         16913
3         10148
4          1338
          ...  
279787    11114
279788    28440
279789     6671
279790    17667
279791     5611
Name: street, Length: 279792, dtype: object

In [58]:
X_df['street'].fillna(0)

0         27289
1         17052
2         16913
3         10148
4          1338
          ...  
279787    11114
279788    28440
279789     6671
279790    17667
279791     5611
Name: street, Length: 279792, dtype: object

In [59]:
X_df['street'] = pd.to_numeric(X_df['street'])

In [60]:
X_df['street'].replace({0:X_df['street'].median()})

0         27289.0
1         17052.0
2         16913.0
3         10148.0
4          1338.0
           ...   
279787    11114.0
279788    28440.0
279789     6671.0
279790    17667.0
279791     5611.0
Name: street, Length: 279792, dtype: float64

In [61]:
X_df['street']

0         27289.0
1         17052.0
2         16913.0
3         10148.0
4          1338.0
           ...   
279787    11114.0
279788    28440.0
279789     6671.0
279790    17667.0
279791     5611.0
Name: street, Length: 279792, dtype: float64